<a href="https://colab.research.google.com/github/stevengregori92/Learn-PyForFinance/blob/main/Target_Function_with_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install luwiji

In [44]:
!gdown https://drive.google.com/uc?id=1LfMank89K6cKESN0S92FXeFvKT-Rx7BU
!unzip /content/sp500_joined_close.zip

Downloading...
From: https://drive.google.com/uc?id=1LfMank89K6cKESN0S92FXeFvKT-Rx7BU
To: /content/sp500_joined_close.zip
100% 13.8M/13.8M [00:00<00:00, 106MB/s] 
Archive:  /content/sp500_joined_close.zip
replace sp500_joined_close.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: sp500_joined_close.csv  


In [45]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [46]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [47]:
from collections import Counter
import pickle

In [48]:
def process_data_for_labels(ticker):
  hm_days = 7
  df = pd.read_csv('sp500_joined_close.csv', index_col=0)
  tickers = df.columns.values.tolist()
  df.fillna(0, inplace=True)

  for i in range(1,hm_days+1):
    df['{}_{}d'.format(ticker,i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]

  df.fillna(0, inplace=True)
  return tickers, df

In [49]:
def buy_sell_hold(*args):
  cols = [c for c in args]
  requirement = 0.02
  for col in cols:
    if col > requirement:
      return 1
    if col < -requirement:
      return -1
  return 0

In [50]:
def extract_featuresets(ticker):
  tickers, df = process_data_for_labels(ticker)

  df['{}_target'.format(ticker)] = list(map(buy_sell_hold,
                                        df['{}_1d'.format(ticker)],
                                        df['{}_2d'.format(ticker)],
                                        df['{}_3d'.format(ticker)],
                                        df['{}_4d'.format(ticker)],
                                        df['{}_5d'.format(ticker)],
                                        df['{}_6d'.format(ticker)],
                                        df['{}_7d'.format(ticker)]))

  vals = df['{}_target'.format(ticker)].values.tolist()
  str_vals = [str(i) for i in vals]
  print('Data spread: ', Counter(str_vals))

  df.fillna(0, inplace=True)
  df = df.replace([np.inf, -np.inf], np.nan)
  df.dropna(inplace=True)

  df_vals = df[[ticker for ticker in tickers]].pct_change()
  df_vals = df_vals.replace([np.inf, -np.inf], 0)
  df_vals.fillna(0, inplace=True)

  X = df_vals.values
  y = df['{}_target'.format(ticker)].values

  # print(y)
  return X , y, df

In [51]:
def do_ml(ticker):
  X, y, df = extract_featuresets(ticker)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  clf = VotingClassifier([('rf', RandomForestClassifier()),
                          ('svm', SVC())])

  clf.fit(X_train, y_train)
  confidence = clf.score(X_test, y_test)
  print('Accuracy: ', confidence)
  predictions = clf.predict(X_test)
  print('Predicted spread: ', Counter(predictions))

  return confidence

In [52]:
do_ml('AAPL')

Data spread:  Counter({'1': 2098, '-1': 1830, '0': 349})
Accuracy:  0.5
Predicted spread:  Counter({1: 490, -1: 366})


0.5